In [1]:
# 数值稳定性
# 神经网络的梯度
# 考虑如下有d层的神经网络
# h^t = f_t(h^t-1)  and  y = l * f_d * ... * f_1(x); 这里的t表示层
# 假设h^(t-1)是第t-1的隐藏层的输出, 经过一个f_t, 得到我们第t层的输出
# y可以表示成为x进来, 然后从f_1第一层, 一直到f_d第d层, 最后到一个损失函数, 就是我们要进行优化的目标函数

# 计算损失l关于W_t的梯度
# dl/dW^t = dl/dh^d * dh^d/dh^(d-1) * ... * dh^(t+1)/dh^t * dh^t/dW^t
# 这里dh^d/dh^(d-1) * ... * dh^(t+1)/dh^t 是d-t次的矩阵乘法

# 数值稳定的两个常见问题: 数据爆炸 梯度消失
# 数据爆炸: 假设1.5^100 = 4*10^17
# 梯度消失: 0.8^100 = 2*10^-10

# MLP
# 加入如下MLP(为简单省略了偏移)
# f_t(h^t-1) = sign(W^t * h^t-1) sign是激活函数
# 这里h^t-1是第t层的输入, 也就是第t-1层的输出, 可以表示为第t层的权重 * 输入

# dh^t/dh^(t-1) = diag(sign'(W^t * h^(t-1)))(W^t)^T


In [2]:
# 梯度爆炸
# 使用ReLu作为激活函数
# sign(x) = max(0, x) and sign'(x) = 1 if x>0; = if x<0
# 这样 (d-1)pai(i=t) dh^(i+1)/dh^i = iag(sign'(W^t * h^(t-1)))(W^t)^T的一些元素来源于 (d-1)pai(i=t) (W^i)^T
#   - 如果d-t很大, 值将会很大

# 梯度爆炸的问题
# 值超出值域(infinity)
#   - 对于16位浮点数尤为严重
# 对学习率敏感
#   - 如果学习率太大 -> 大参数 ->更大的梯度
#   - 如果学习率太小 -> 小参数 ->训练无进展
# 我们可能需要在训练中不断调整学习率

# 梯度消失
# 使用sigmoid作为激活函数
# sign(x) = 1/1+e^-x, sign'(x) = sign(x) * (1 - sign(x))
# 这样 (d-1)pai(i=t) dh^(i+1)/dh^i = iag(sign'(W^t * h^(t-1)))(W^t)^T的元素值就是d-t个小数值的乘积

# 梯度消失的问题
# 梯度值变成0
#   - 对16为浮点数尤为严重
# 训练没有进展
#   - 不管如何选择学习率
# 对于底部层尤为严重
#   - 仅仅顶部层的训练较好
#   - 无法让网络更深

# 总结：当数值太大或者太小的时候, 都会导致数值问题; 常发生在深度模型中, 因为其会对n个数累乘

In [ ]:
# 让训练更加稳定
# 目标: 让梯度值在合理的范围, 例如: [1e-6, 1e3]
# 将乘法变成加法, 例如: ResNet, LSTM
# 归一化, 梯度归一化, 梯度剪裁
# 合理的权重初始和激活函数

# 让每层的方差是一个常数
#   - 将每层的输出和梯度都看做随机变量
#   - 将他们的均值和方差都保持一致
# 正向 E[h^t _i] = 0  Var[h^t _i] = a
# 反向 E[dl/dh^t _i]= 0 Var[dl/dh^t _i] =b
# 这里a, b都是常数

# 权重初始化
#  - 在合理值区间里随机初始参数
#  - 训练开始的时候更容易有数值不稳定
#      - 远离最优解的地方损失函数表面函数可能很复杂
#      - 最优解附近表面会比较平
#  - 使用 N(0, 0.01)来初始可能对小网络没问题, 但不能保证深度神经网络

# 例子: MLP
#  - 假设 
#    - w^t _i,j 是i.i.d, 那么E[w^t _i,j] = 0, Var[w^t _i,j] = gamma_t
#    - h^(t-1) _i 独立于 w^t_i,j
#  - 假设没有激活函数 h^t = W^t * h^(t-1), 这里 W^t in R^(n_t * n_t-1)
#  E[h^t _i] = E[sigma_j w^t _i,j h^(t-1) _j] = sigma_j E[w^t _i,j][h^(t-1) _j] = 0

# 正向方差
# Var[h^t _i] = E[(h^t _i)^2] - E[h^t _i]^2 = E([sigma_j w^t _i,j h^(t-1) _j]^2) 最后 = n*gamma_t = 1

# Xavier 初始
# 难以需要满足n_(t-1) gamma_t = 1和n_t gamma_t = 1
# Xavier 使得 gamma_t(n_(t-1)/2 = 1 -> gamma_t = 2/(n_(t-1) + n_t)
#    - 正太分布 N(0, sqrt(2/(n_(t-1) + n_t)))
#    - 均匀分布 U(-sqrt(6/(n_(t-1) + n_t)), sqrt(6/(n_(t-1) + n_t)))
#       - 分布 U[-a, a] 和方差a^2/3
#    - 适配权重形状变化, 特别是n_t

# 总结: 合理的权重初始值和激活函数的选取可以提高数值稳定性